# Базовое решение для задачи A 

In [3]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier

Читаем входные файлы с данными

In [4]:
transactions = pd.read_csv('./transactions.csv')
customers_gender = pd.read_csv('./customers_gender_train.csv')

In [5]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


Для каждого клиента подсчитываем различные `mcc_code`

In [11]:
X = transactions.groupby('customer_id', axis = 1)[['mcc_code']].apply(lambda x: x.unstack().value_counts()).unstack().fillna(0)

In [29]:
X = (transactions.groupby('customer_id')
     .apply(lambda x: x[['mcc_code']].unstack().value_counts())
     .unstack()
     .fillna(0))

In [33]:
X.sum().sum()

6849346.0

In [34]:
customers_gender = customers_gender.set_index('customer_id')

In [35]:
Y_train = customers_gender.loc[X.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [36]:
X_train = X.reset_index()
X_train = X_train.loc[Y_train.index].set_index('customer_id')

Обучаемся на всех данных

In [37]:
clf = GradientBoostingClassifier(random_state=13)
clf.fit(X_train, Y_train)

/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=13,
              subsample=1.0, verbose=0, warm_start=False)

Предсказываем пол для тестовых данных и создаём файл с ответом

In [38]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

In [39]:
result.to_csv('baseline_a.csv', index=False)

In [40]:
X.shape

(15000, 184)